In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas

In [2]:
df = pd.read_csv('owid-co2-data.csv')

In [3]:
df.head(5)

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1750,AFG,2802560.0,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1751,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1752,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1753,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1754,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
type(df['country'] == 'World')

pandas.core.series.Series

In [5]:
#df = df[ (df['country'] == 'World') & (df['year'] >= 2010) ]
df = df[df['year'] >= 2000]

In [6]:
#returns blanks for some reason
# df = df.query(" country == 'world' & year >= 2010 ")

In [7]:
type(df[df['country'] == 'World'])

pandas.core.frame.DataFrame

In [8]:
df.shape

(6120, 79)

In [9]:
df.year.min()

2000

### (0) Data pre-processing

In [10]:
type(df)

pandas.core.frame.DataFrame

In [11]:
#replace na with 0
df['gdp_per_capita'] = np.where(df['population'] != 0 , df['gdp'] / df['population'] , 0)
df = df.fillna(0)

In [12]:
df.shape

(6120, 80)

In [13]:
# make DataFrame pipeline interactive
idf = df.interactive()

### (1) CO2 emission over time by continent

In [14]:
#define panel widgets
year_slider = pn.widgets.IntSlider(name = 'Year Slider', start = df['year'].min(), end= df['year'].max(), step = 5, value = df['year'].max())

In [15]:
year_slider

IntSlider(end=2023, name='Year Slider', start=2000, step=5, value=2023)

In [16]:
# Radio buttons for CO2 Measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name = 'Y Axis',
    options = ['co2', 'co2_per_capita'],
    button_type = 'success'
)

In [17]:
yaxis_co2

RadioButtonGroup(button_type='success', name='Y Axis', options=['co2', 'co2_per_capita'], value='co2')

## Connecting data pipeline to widgets

In [18]:
continents = ['world', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

In [19]:
co2_pipeline = (
    idf[
    (idf.year <= year_slider) &
    (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'year')
    .reset_index(drop = True)
)

In [20]:
type(idf[(idf.year <= year_slider) & (idf.country.isin(continents))])

hvplot.interactive.Interactive

In [21]:
co2_plot = co2_pipeline.hvplot(x = 'year', by = 'country', y = yaxis_co2, kind = 'line', line_width=2, title = "CO2 emission by continent")

In [22]:
co2_plot

## 2) Table - Co2 emission over time by continent

In [23]:
# since co2_pipeline is being used, all widgets will be applicable here. 
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination = 'remote', page_size = 10, sizing_mode = 'stretch_width')

In [24]:
co2_table

## 3) CO2 vs GDP scatterplot

In [25]:
co2_vs_gdp_scatterplot_pipeline = (
    idf[
    (idf['year'] == year_slider) &
    (  ~ (idf['country'].isin(continents)))
    ]
    .groupby(['country','year','gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'year')
    .reset_index(drop = True)
)

In [26]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x = 'gdp_per_capita', y = 'co2', by = 'country', size = 88, kind = 'scatter', legend = False, height = 500, width = 500)

In [27]:
co2_vs_gdp_scatterplot

## 4) Bar chart with CO2 sources by continents

In [28]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['coal_co2', 'oil_co2', 'gas_co2'],
    button_type = 'success'
)

In [29]:
yaxis_co2_source

RadioButtonGroup(button_type='success', name='Y axis', options=['coal_co2', 'oil_co2', ...], value='coal_co2')

In [30]:
continents_excl_world = ['Asia', 'oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

In [31]:
co2_source_bar_pipeline = (
    idf[
    (idf.year == year_slider) &
    (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by = 'year')
    .reset_index(drop = True)
)

In [32]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind = 'bar', x = 'country', y = yaxis_co2_source, title = 'CO2 by continent')

In [33]:
co2_source_bar_plot

## Creating Dashboard

In [34]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='World CO2 emission dashboard', 
    sidebar=[pn.pane.Markdown("# CO2 Emissions and Climate Change"), 
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
             pn.pane.PNG('climate_day.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, co2_plot.panel(width=700), margin=(0,25)), co2_table.panel(width=500)), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
template.show()
# template.servable();

Launching server at http://localhost:58136
